In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym

import models
import payoffs
import plotting
import selection_functions
import symbolics

In [2]:
%matplotlib inline
plt.style.use("ggplot")

# One-locus model

Start from full model. Set $x_3=x_4=0, x_1, x_2=1 - x1$. Monomorphic with $\gamma=G$ only.

In [3]:
x1 = sym.symbols('x1', real=True, positive=True)
T, R, P, S = sym.symbols('T, R, P, S', real=True, positive=True)
M = sym.symbols("M", real=True, positive=True)
epsilon = sym.symbols("epsilon", real=True, positive=True)

UGA = symbolics.UGA
UgA = symbolics.UgA

# Total offspring (fitness)

In [4]:
x = np.array([[x1], [1-x1], [0], [0]])
payoff_kernel = np.array([[R, S], [T, P]])
W = models.generalized_sexual_selection(x, UGA, UgA, payoff_kernel, M=M, m=0, mutation_rate=epsilon)
N, = models.total_offspring(W, x)

In [5]:
N = sym.simplify(N)

In [6]:
sym.factor(N, UGA(x1))

4.0*(-0.5*M + 0.5*P + (-1.0*P + 0.5*S + 0.5*T)*UGA(x1) + (0.5*P + 0.5*R - 0.5*S - 0.5*T)*UGA(x1)**2)

After a bit of algebra we arrive at the following expression for total offspring.

\begin{align}
   % N(x_1) =& 2.0*P*UGA(x1)**2 - 4.0*P*UGA(x1) + 2.0*P + 2.0*R*UGA(x1)**2 - 2.0*S*UGA(x1)**2 + 2.0*S*UGA(x1) - 2.0*T*UGA(x1)**2 + 2.0*T*UGA(x1)\\
   % N(x_1) =& 2\bigg(PU_{GA}(x_1)^2 - 2PU_{GA}(x_1) + P + RU_{GA}(x_1)^2 - SU_{GA}(x_1)^2 + SU_{GA}(x_1) - TU_{GA}(x_1)^2 + TU_{GA}(x_1)\bigg) \\
   % N(x_1) =& 2\bigg(RU_{GA}(x_1)^2 + PU_{GA}(x_1)^2 - TU_{GA}(x_1)^2 - SU_{GA}(x_1)^2 + SU_{GA}(x_1) + TU_{GA}(x_1) - 2PU_{GA}(x_1) + P\bigg) \\
    N(x_1) =& 2\bigg(\big((R + P) - (T + S)\big)U_{GA}(x_1)^2 + \big((T + S) - 2P\big)U_{GA}(x_1) + P\bigg) \\
\end{align}

# The locus of potential equilibria (LPE)

In [7]:
(f,), _, _, _ = models.offspring_genotypes_evolution(W, x)

In [8]:
# solving for the LPE using Python
x1_star, = sym.solve(f, x1, implicit=True)
x1_star =  sym.cancel(x1_star)

$$ \frac{2\big(U_G(S - M) + (R - S)U_G^2\big)}{F(U_G)} $$

In [9]:
x1_star

-1.0*(4.0*M*epsilon*UGA(x1) - 4.0*M*epsilon - 2.0*M*UGA(x1) + 4.0*P*epsilon*UGA(x1)**2 - 8.0*P*epsilon*UGA(x1) + 4.0*P*epsilon + 2.0*R*UGA(x1)**2 - 2.0*S*UGA(x1)**2 + 2.0*S*UGA(x1) - 4.0*T*epsilon*UGA(x1)**2 + 4.0*T*epsilon*UGA(x1))/(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon*UGA(x1)**2 + 8.0*P*epsilon*UGA(x1) - 4.0*P*epsilon - 2.0*P*UGA(x1)**2 + 4.0*P*UGA(x1) - 2.0*P - 4.0*R*epsilon*UGA(x1)**2 - 2.0*R*UGA(x1)**2 + 4.0*S*epsilon*UGA(x1)**2 - 4.0*S*epsilon*UGA(x1) + 2.0*S*UGA(x1)**2 - 2.0*S*UGA(x1) + 4.0*T*epsilon*UGA(x1)**2 - 4.0*T*epsilon*UGA(x1) + 2.0*T*UGA(x1)**2 - 2.0*T*UGA(x1))

In [124]:
x1_star = sym.factor(x1_star.subs({UGA(x1): 1}))

In [157]:
x1_star.subs({UGA(x1): 1, M: 0})

0.5/(1.0*epsilon + 0.5)

# Invadability

Restrict the full model to obtain a monomorphic $\gamma$ equilibrium, add a small amount of individuals carrying the $g$ allele of the $\gamma$ gene and simulate.

In [11]:
x1, x2, x3 = sym.symbols("x1, x2, x3", real=True, positive=True)
x4 = 1 - x1 - x2 - x3
xA, xG = sym.symbols("x_A, x_G", real=True, positive=True)
x = np.array([[x1], [x2], [x3], [x4]])
W = models.generalized_sexual_selection(x, UGA, UgA, payoff_kernel, M=M, mutation_rate=epsilon)
f1, f2, f3, _ = models.offspring_genotypes_evolution(W, x)

In [12]:
F = sym.Matrix([f1, f2, f3])

In [13]:
F_jac = F.jacobian([x1, x2, x3])

In [206]:
sym.simplify(x1_star.subs({UGA(x1): UG}))

(4.0*M*UG*epsilon - 2.0*M*UG - 4.0*M*epsilon + 4.0*P*UG**2*epsilon - 8.0*P*UG*epsilon + 4.0*P*epsilon + 2.0*R*UG**2 - 2.0*S*UG**2 + 2.0*S*UG - 4.0*T*UG**2*epsilon + 4.0*T*UG*epsilon)/(-4.0*M*epsilon - 2.0*M + 4.0*P*UG**2*epsilon + 2.0*P*UG**2 - 8.0*P*UG*epsilon - 4.0*P*UG + 4.0*P*epsilon + 2.0*P + 4.0*R*UG**2*epsilon + 2.0*R*UG**2 - 4.0*S*UG**2*epsilon - 2.0*S*UG**2 + 4.0*S*UG*epsilon + 2.0*S*UG - 4.0*T*UG**2*epsilon - 2.0*T*UG**2 + 4.0*T*UG*epsilon + 2.0*T*UG)

In [14]:
UG = sym.symbols("UG", positive=True, real=True)
Ug = sym.symbols("Ug", positive=True, real=True)
UG_prime = sym.symbols("UG_prime", positive=True, real=True)

evaluated_F_jac = sym.simplify(F_jac.subs({x2: 1 - x1, x3:0})
                                    .doit()
                                    .subs({UGA(x1): UG, UgA(x1): Ug, sym.Derivative(UGA(x1), x1): UG_prime}))
 

In [16]:
eigenvals = evaluated_F_jac.eigenvals()

KeyboardInterrupt: 

In [ ]:
e1, e2, e3 = eigenvals.keys()

In [187]:
x1_star.subs({UGA(x1): UG})

0.5*(M - R)/((1.0*M - 1.0*R)*(1.0*epsilon + 0.5))

In [142]:
UG = sym.symbols("UG", positive=True, real=True)
Ug = sym.symbols("Ug", positive=True, real=True)
UG_prime = sym.symbols("UG_prime", positive=True, real=True)

evaluated_F_jac = sym.simplify(F_jac.subs({x2: 1 - x1, x3:0})
           .doit()
           .subs({UGA(x1): UG, UgA(x1): Ug, sym.Derivative(UGA(x1), x1): UG_prime})
           .subs({x1: x1_star.subs({UGA(x1): UG})})
           .subs({Ug: x1_star.subs({UGA(x1): UG})})
           .subs({UG: 1}))

In [143]:
eigenvals = evaluated_F_jac.eigenvals()

In [144]:
e1, e2, e3 = eigenvals.keys()

In [145]:
e1

-(2*M*UG_prime*epsilon - M*UG_prime + 2*M*epsilon + M - 2*R*epsilon - R - 2*T*UG_prime*epsilon + T*UG_prime)/(2*M - 2*R)

In [155]:
e2.subs({M: 0})

-(-8*P*epsilon**3 + 12*P*epsilon**2 - 24*R*epsilon**2 - 26*R*epsilon - 3*R - 4*S*epsilon**2 + 6*S*epsilon - 4*T*epsilon**2 + 6*T*epsilon + 2*epsilon*sqrt(16*P**2*epsilon**4 + 128*P*R*epsilon**4 + 32*P*R*epsilon**3 - 8*P*R*epsilon**2 + 16*P*S*epsilon**3 + 16*P*T*epsilon**3 + 16*R**2*epsilon**2 + 24*R**2*epsilon + 9*R**2 + 16*R*S*epsilon**2 + 12*R*S*epsilon + 64*R*T*epsilon**3 + 16*R*T*epsilon**2 - 4*R*T*epsilon + 4*S**2*epsilon**2 + 8*S*T*epsilon**2 + 4*T**2*epsilon**2) + sqrt(16*P**2*epsilon**4 + 128*P*R*epsilon**4 + 32*P*R*epsilon**3 - 8*P*R*epsilon**2 + 16*P*S*epsilon**3 + 16*P*T*epsilon**3 + 16*R**2*epsilon**2 + 24*R**2*epsilon + 9*R**2 + 16*R*S*epsilon**2 + 12*R*S*epsilon + 64*R*T*epsilon**3 + 16*R*T*epsilon**2 - 4*R*T*epsilon + 4*S**2*epsilon**2 + 8*S*T*epsilon**2 + 4*T**2*epsilon**2))/(-32*R*epsilon**2 - 32*R*epsilon - 8*R)

In [175]:
str(e3.subs({M: 0}))

'-(-8*P*epsilon**3 + 12*P*epsilon**2 - 24*R*epsilon**2 - 26*R*epsilon - 3*R - 4*S*epsilon**2 + 6*S*epsilon - 4*T*epsilon**2 + 6*T*epsilon - 2*epsilon*sqrt(16*P**2*epsilon**4 + 128*P*R*epsilon**4 + 32*P*R*epsilon**3 - 8*P*R*epsilon**2 + 16*P*S*epsilon**3 + 16*P*T*epsilon**3 + 16*R**2*epsilon**2 + 24*R**2*epsilon + 9*R**2 + 16*R*S*epsilon**2 + 12*R*S*epsilon + 64*R*T*epsilon**3 + 16*R*T*epsilon**2 - 4*R*T*epsilon + 4*S**2*epsilon**2 + 8*S*T*epsilon**2 + 4*T**2*epsilon**2) - sqrt(16*P**2*epsilon**4 + 128*P*R*epsilon**4 + 32*P*R*epsilon**3 - 8*P*R*epsilon**2 + 16*P*S*epsilon**3 + 16*P*T*epsilon**3 + 16*R**2*epsilon**2 + 24*R**2*epsilon + 9*R**2 + 16*R*S*epsilon**2 + 12*R*S*epsilon + 64*R*T*epsilon**3 + 16*R*T*epsilon**2 - 4*R*T*epsilon + 4*S**2*epsilon**2 + 8*S*T*epsilon**2 + 4*T**2*epsilon**2))/(-32*R*epsilon**2 - 32*R*epsilon - 8*R)'

In [159]:
delta = (64*P**2*epsilon**4 + 
         32*P*R*epsilon**2 +
         64*P*S*epsilon**3 +
         64*P*T*epsilon**3 +
         4*R**2 +
         16*R*S*epsilon + 
         16*R*T*epsilon +
         16*S**2*epsilon**2 +
         32*S*T*epsilon**2 +
         16*T**2*epsilon**2 +
         
         128*P*R*epsilon**4 + 
         32*P*R*epsilon**3 -  
         16*R**2*epsilon**2 + 
         24*R**2*epsilon + 
         8*R**2 + 
         16*R*S*epsilon**2 +  
         64*R*T*epsilon**3 + 
         16*R*T*epsilon**2 +
         8*R*S*epsilon -    
         48*P**2*epsilon**4 - 
         24*P*R*epsilon**2 - 
         48*P*S*epsilon**3 - 
         48*P*T*epsilon**3 - 
         3*R**2 -
         12*R*S*epsilon - 
         12*R*T*epsilon - 
         12*S**2*epsilon**2 - 
         24*S*T*epsilon**2 - 
         12*T**2*epsilon**2)


In [176]:
A = -8*P*epsilon**3 + 12*P*epsilon**2 - 24*R*epsilon**2 - 26*R*epsilon - 3*R - 4*S*epsilon**2 + 6*S*epsilon - 4*T*epsilon**2 + 6*T*epsilon

In [178]:
A / (2*epsilon + 1)**2

(-8*P*epsilon**3 + 12*P*epsilon**2 - 24*R*epsilon**2 - 26*R*epsilon - 3*R - 4*S*epsilon**2 + 6*S*epsilon - 4*T*epsilon**2 + 6*T*epsilon)/(2*epsilon + 1)**2

In [174]:
n = (128*P*R*epsilon**4 + 
         32*P*R*epsilon**3 -  
         #16*R**2*epsilon**2 + 
         8*R**2*epsilon + 
         4*R**2 + 
         16*R*S*epsilon**2 +  
         64*R*T*epsilon**3 + 
         16*R*T*epsilon**2 +
         8*R*S*epsilon)
d = (2*epsilon + 1)**4
sym.simplify(n / d)

4*R*(32*P*epsilon**4 + 8*P*epsilon**3 - 2*R*epsilon + R + 4*S*epsilon**2 + 2*S*epsilon + 16*T*epsilon**3 + 4*T*epsilon**2)/(2*epsilon + 1)**4

In [179]:
K = (128*P*R*epsilon**4 + 
     32*P*R*epsilon**3 -  
     8*R**2*epsilon + 
     4*R**2 + 
     16*R*S*epsilon**2 +  
     64*R*T*epsilon**3 + 
     16*R*T*epsilon**2 +
     8*R*S*epsilon)

In [181]:
sym.simplify(-K - A)

-128*P*R*epsilon**4 - 32*P*R*epsilon**3 + 8*P*epsilon**3 - 12*P*epsilon**2 + 8*R**2*epsilon - 4*R**2 - 16*R*S*epsilon**2 - 8*R*S*epsilon - 64*R*T*epsilon**3 - 16*R*T*epsilon**2 + 24*R*epsilon**2 + 26*R*epsilon + 3*R + 4*S*epsilon**2 - 6*S*epsilon + 4*T*epsilon**2 - 6*T*epsilon

In [140]:
_epsilon = 4e-1
e2.subs({T: 4, R: 2, P: 1, S: 0.5, M: 0, epsilon: _epsilon}) < 0

True

In [141]:
e3.subs({T: 4, R: 2, P: 1, S: 0.5, M: 0, epsilon: _epsilon}) < 0

True

In [162]:
N, = models.total_offspring(W, x)

In [167]:
sym.expand(sym.simplify(N.subs({x2: 1 - x1, x3: 0}).subs({UGA(x1): x1_star.subs({UGA(x1): 1, M: 0})}).subs({M: 0}))**2)

4.0*P**2*epsilon**4/(1.0*epsilon**4 + 2.0*epsilon**3 + 1.5*epsilon**2 + 0.5*epsilon + 0.0625) + 2.0*P*R*epsilon**2/(1.0*epsilon**4 + 2.0*epsilon**3 + 1.5*epsilon**2 + 0.5*epsilon + 0.0625) + 4.0*P*S*epsilon**3/(1.0*epsilon**4 + 2.0*epsilon**3 + 1.5*epsilon**2 + 0.5*epsilon + 0.0625) + 4.0*P*T*epsilon**3/(1.0*epsilon**4 + 2.0*epsilon**3 + 1.5*epsilon**2 + 0.5*epsilon + 0.0625) + 0.25*R**2/(1.0*epsilon**4 + 2.0*epsilon**3 + 1.5*epsilon**2 + 0.5*epsilon + 0.0625) + 1.0*R*S*epsilon/(1.0*epsilon**4 + 2.0*epsilon**3 + 1.5*epsilon**2 + 0.5*epsilon + 0.0625) + 1.0*R*T*epsilon/(1.0*epsilon**4 + 2.0*epsilon**3 + 1.5*epsilon**2 + 0.5*epsilon + 0.0625) + 1.0*S**2*epsilon**2/(1.0*epsilon**4 + 2.0*epsilon**3 + 1.5*epsilon**2 + 0.5*epsilon + 0.0625) + 2.0*S*T*epsilon**2/(1.0*epsilon**4 + 2.0*epsilon**3 + 1.5*epsilon**2 + 0.5*epsilon + 0.0625) + 1.0*T**2*epsilon**2/(1.0*epsilon**4 + 2.0*epsilon**3 + 1.5*epsilon**2 + 0.5*epsilon + 0.0625)

In [139]:
sym.simplify(N.subs({x2: 1 - x1, x3: 0}).subs({UGA(x1): x1})).subs({x1: x1_star}).subs({M: 0, P: 1, S: 1 / R, T: 2 * R}).subs({R: 2}).subs({epsilon: 4e-1})

3.85185185185185

In [107]:
N.subs({T: 4, R: 2, P: 1, S: 0.5, M: 0, epsilon: 1e-2, UGA(x1): x1_star.subs({UGA(x1): 1})})

x1*(0.005*x3*(3.0*UGA(x1 + x3) + 1.0)*UGA(x1 + x3)/(x1 + x3) + 0.25*(1 - UGA(x1 + x3))*(6*UGA(x1 + x3) + 2)*(-x1 - x2 - x3 + 1)/(-x1 - x3 + 1)) + x1*(0.495*x3*(3.0*UGA(x1 + x3) + 1.0)*UGA(x1 + x3)/(x1 + x3) + 0.25*(1 - UGA(x1 + x3))*(6*UGA(x1 + x3) + 2)*(-x1 - x2 - x3 + 1)/(-x1 - x3 + 1)) + x1*(0.01*x1*(3.0*UGA(x1 + x3) + 1.0)*UGA(x1 + x3)/(x1 + x3) + 0.5*x2*(1 - UGA(x1 + x3))*(6*UGA(x1 + x3) + 2)/(-x1 - x3 + 1) + 0.005*x3*(3.0*UGA(x1 + x3) + 1.0)*UGA(x1 + x3)/(x1 + x3) + 0.25*(1 - UGA(x1 + x3))*(6*UGA(x1 + x3) + 2)*(-x1 - x2 - x3 + 1)/(-x1 - x3 + 1)) + x1*(0.99*x1*(3.0*UGA(x1 + x3) + 1.0)*UGA(x1 + x3)/(x1 + x3) + 0.5*x2*(1 - UGA(x1 + x3))*(6*UGA(x1 + x3) + 2)/(-x1 - x3 + 1) + 0.495*x3*(3.0*UGA(x1 + x3) + 1.0)*UGA(x1 + x3)/(x1 + x3) + 0.25*(1 - UGA(x1 + x3))*(6*UGA(x1 + x3) + 2)*(-x1 - x2 - x3 + 1)/(-x1 - x3 + 1)) + x2*(0.25*x3*(3.0*UGA(x1 + x3) + 1.0)*UGA(x1 + x3)/(x1 + x3) + 0.005*(1 - UGA(x1 + x3))*(6*UGA(x1 + x3) + 2)*(-x1 - x2 - x3 + 1)/(-x1 - x3 + 1)) + x2*(0.25*x3*(3.0*UGA(x1 + 

In [114]:
import pickle

with open('non-invadable-eigenvalues.pickle', 'wb') as handle:
    pickle.dump(eigenvals, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'eigenvals' is not defined

In [38]:
e1, e2, e3 = eigenvals.keys()

In [42]:
numerator, denominator = sym.fraction(sym.factor(e1))

In [46]:
denominator

2*(P*UGA_star**2 - 2*P*UGA_star + P + R*UGA_star**2 - S*UGA_star**2 + S*UGA_star - T*UGA_star**2 + T*UGA_star)**2

In [48]:
upper_bound, = sym.solve(numerator, UGA_prime_star)

In [50]:
sym.factor(upper_bound)

(2*epsilon + 1)*(P*UGA_star**2 - 2*P*UGA_star + P + R*UGA_star**2 - S*UGA_star**2 + S*UGA_star - T*UGA_star**2 + T*UGA_star)**2/((2*epsilon - 1)*(2*P*R*UGA_star**2 - 2*P*R*UGA_star - P*S*UGA_star**2 + 2*P*S*UGA_star - P*S - R*T*UGA_star**2))

In [95]:
e1.subs({UGA_star: 1, UGA_prime_star: 0})

-(2*R**2*epsilon + R**2)/(2*R**2)

In [73]:
# vectorized numeric repr for the eigenvalue
_numeric_e2 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, epsilon),
                           e2.subs({x1: x1_star}),
                           modules="numpy")

In [75]:
def plot_second_eigenvalue(T, R, P, S, epsilon):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_2$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e2(UGAs, UgAs, T, R, P, S, epsilon)
    cax = ax.imshow(Z, origin="lower")
    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [80]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

# slider used to control the mutation rate
e_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=1e-3, description=r"$\epsilon$", readout_format=".3f")

w = widgets.interactive(plot_second_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, epsilon=e_slider)
display(w)

A Jupyter Widget

In [105]:
evaluated = (e2.subs({x1: x1_star, UgA_star: x1_star})
               .subs({UGA_star: 1, UGA_prime_star: 0}))
together = sym.together(evaluated)

In [107]:
expanded = sym.expand(together)

In [108]:
expanded

-4.0*P*epsilon**4/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) + 6.0*P*epsilon**3/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) - 12.0*R*epsilon**3/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) - 13.0*R*epsilon**2/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) - 1.5*R*epsilon/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) - 2.0*S*epsilon**3/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) + 3.0*S*epsilon**2/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) - 2.0*T*epsilon**3/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) + 3.0*T*epsilon**2/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) + 0.015625*sqrt(65536.0*P**2*epsilon**8 + 65536.0*P**2*epsilon**7 + 16384.0*P**2*epsilon**6 + 524288.0*P*R*epsilon**8 + 655360.0*P*R*epsilon**7 + 229376.0*P*R*epsilon**6 - 8192.0*P*R*epsilon**4 + 65536.0*P*S*epsilon**7 + 65536.0*P*S*epsilon**6 + 16384.0*P*S*epsilon**5 + 65536.0*P*T*epsilon**7 + 65536.0*P*T*epsilon**6 + 16384.

In [110]:
tmp_expr = sym.together(expanded)

In [111]:
tmp_expr

(-4.0*P*epsilon**3 + 6.0*P*epsilon**2 - 12.0*R*epsilon**2 - 13.0*R*epsilon - 1.5*R - 2.0*S*epsilon**2 + 3.0*S*epsilon - 2.0*T*epsilon**2 + 3.0*T*epsilon + 0.015625*sqrt(65536.0*P**2*epsilon**6 + 65536.0*P**2*epsilon**5 + 16384.0*P**2*epsilon**4 + 524288.0*P*R*epsilon**6 + 655360.0*P*R*epsilon**5 + 229376.0*P*R*epsilon**4 - 8192.0*P*R*epsilon**2 + 65536.0*P*S*epsilon**5 + 65536.0*P*S*epsilon**4 + 16384.0*P*S*epsilon**3 + 65536.0*P*T*epsilon**5 + 65536.0*P*T*epsilon**4 + 16384.0*P*T*epsilon**3 + 65536.0*R**2*epsilon**4 + 163840.0*R**2*epsilon**3 + 151552.0*R**2*epsilon**2 + 61440.0*R**2*epsilon + 9216.0*R**2 + 65536.0*R*S*epsilon**4 + 114688.0*R*S*epsilon**3 + 65536.0*R*S*epsilon**2 + 12288.0*R*S*epsilon + 262144.0*R*T*epsilon**5 + 327680.0*R*T*epsilon**4 + 114688.0*R*T*epsilon**3 - 4096.0*R*T*epsilon + 16384.0*S**2*epsilon**4 + 16384.0*S**2*epsilon**3 + 4096.0*S**2*epsilon**2 + 32768.0*S*T*epsilon**4 + 32768.0*S*T*epsilon**3 + 8192.0*S*T*epsilon**2 + 16384.0*T**2*epsilon**4 + 16384.0*T*

In [117]:
tmp_expr.subs({epsilon**2: 0})

(-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon + 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0))

\begin{align}
   % (-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon + 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0)) < 0 \\
    \frac{(-13R\epsilon - 1.5R + 3S\epsilon + 3T\epsilon + 0.015625\sqrt{61440R^2\epsilon + 9216R^2 + 12288RS\epsilon - 4096RT\epsilon}}{R(16\epsilon + 4.0)} < 0 \\
    \frac{(-13R\epsilon - \frac{3}{2}R + 3S\epsilon + 3T\epsilon + \frac{1}{64}\sqrt{61440R^2\epsilon + 9216R^2 + 12288RS\epsilon - 4096RT\epsilon}}{R(16\epsilon + 4.0)} < 0 \\
    \frac{(-13R\epsilon - \frac{3}{2}R + 3S\epsilon + 3T\epsilon + \frac{1}{2}\sqrt{60R^2\epsilon + 9R^2 + 12RS\epsilon - 4RT\epsilon}}{R(16\epsilon + 4)} < 0
\end{align}

In [60]:
numerator, denominator = sym.fraction(sym.factor(e2))

In [61]:
denominator

8*x1*(x1 - 1)*(P*UGA_star**2 - 2*P*UGA_star + P + R*UGA_star**2 - S*UGA_star**2 + S*UGA_star - T*UGA_star**2 + T*UGA_star)

In [77]:
# vectorized numeric repr for the eigenvalue
_numeric_e3 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, epsilon),
                           e3.subs({x1: x1_star}),
                           modules="numpy")

In [78]:
def plot_third_eigenvalue(T, R, P, S, epsilon):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_3$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e3(UGAs, UgAs, T, R, P, S, epsilon)
    cax = ax.imshow(Z, origin="lower")
    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [79]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

# slider used to control the mutation rate
e_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=1e-3, description=r"$\epsilon$", readout_format=".3f")

w = widgets.interactive(plot_third_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, epsilon=e_slider)
display(w)

A Jupyter Widget

In [158]:
e2 == e3

False

In [164]:
sym.simplify(e2.subs({x1: x1_star, UgA_star: x1_star}).subs({UGA_star: 1, UGA_prime_star: 0})).subs({epsilon**2: 0})

(-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon + 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0))

In [165]:
sym.simplify(e3.subs({x1: x1_star, UgA_star: x1_star}).subs({UGA_star: 1, UGA_prime_star: 0})).subs({epsilon**2: 0})

(-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon - 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0))

In [153]:
evaluated = (e3.subs({x1: x1_star, UgA_star: x1_star})
               .subs({UGA_star: 1, UGA_prime_star: 0}))
together = sym.together(evaluated)

In [154]:
expanded = sym.expand(together)

In [ ]:
tmp_expr = sym.together(expanded)

In [155]:
tmp_expr.subs({epsilon**2: 0})

(-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon + 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0))

In [ ]:
(-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon + 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0))

First eigenvalue can be written as a hyperbola in $U_{gA}^*,U_{GA}^*$ plane.

$$ e_1 = \left(\frac{\big((R + P) - (T + S)\big)U_{gA}^{*2} + \big((T + S) - 2P\big)U_{gA}^*}{N^*}\right) - \left(\frac{\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^*}{N^*}\right) $$

### Completing the square

To write this hyperbola in standard form we need to [complete the square](https://en.wikipedia.org/wiki/Completing_the_square). Completing the square for the quadratic polynomial in $U_{GA}(x^*)$ yields the following.

\begin{align}
    \big((R + P) - (T + S)\big)\left(U_{GA}^* - \bar{U}_{GA}^*\right)^2 - \frac{1}{2}\big(\bar{N}^* - 2P\big) \\
\end{align}

where $\bar{U}_{GA}^*$ is the value of $U_{GA}^*$ that maximizes total offspring $N$ derived above. Completing the square for the quadratic polynomial in $U_{gA}(x^*)$ yields a similar expression.

\begin{align}
    \big((R + P) - (T + S)\big)\left(U_{gA}^* - \bar{U}_{GA}^*\right)^2 - \frac{1}{2}\big(\bar{N}^* - 2P\big) \\
\end{align}

Substituting these results into the expression for the eigenvalue yields the following.

$$ e_1 = \left(\frac{\big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) - \left(\frac{\big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) $$


### Conditions for negative eigenvalue

Non-invadability requires that this eigenvalue is strictly negative.

$$ \left(\frac{\big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) - \left(\frac{\big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) < 0$$

#### Case: (R + P) < (T + S)
In this case the inequality simplifies to the following.

$$ \big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2} - \big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2} > 0 $$

This implies that we have four sub-cases to consider.

\begin{align}
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) >& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) >& -\big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) >& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) >& -\big(U_{GA}^* - \bar{U}_{GA}^*\big)
\end{align}

These four sub-cases define the following two regions of the $U_{gA}^*,U_{GA}^*$ plane where the eigenvalue is negative.

\begin{align}
   & 2\big(\bar{U}_{GA}^* - U_{gA}^*\big) < U_{GA}^* - U_{gA}^* < 0\\
   & 0 < U_{GA}^* - U_{gA}^* < 2\big(\bar{U}_{GA}^* - U_{gA}\big)
\end{align}

#### Case: (R + P) > (T + S)
In this case the inequality simplifies to the following.

$$ \big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2} - \big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2} < 0 $$

This implies that we again have four sub-cases to consider.

\begin{align}
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) <& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) <& -\big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) <& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) <& -\big(U_{GA}^* - \bar{U}_{GA}^*\big)
\end{align}

These four sub-cases define the following two regions of the $U_{gA}^*,U_{GA}^*$ plane where the eigenvalue is negative.

\begin{align}
   & 2\big(\bar{U}_{GA}^* - U_{gA}^*\big) < U_{GA}^* - U_{gA}^* < 0\\
   & 0 < U_{GA}^* - U_{gA}^* < 2\big(\bar{U}_{GA}^* - U_{gA}\big)
\end{align}


### Asymptotes 

Asymptotes of this hyperbola can be defined as follows.

$$ U_{GA}^* - \bar{U}_{GA}^* = \pm\big(U_{gA}^* - \bar{U}_{GA}^*\big) $$

The above can be written as linear equations in $U_{gA}^*,U_{GA}^*$ plane.

\begin{align}
    U_{GA}^* =& U_{gA}^* \\
    U_{GA}^* =& 2\bar{U}_{GA}^* - U_{gA}^*
\end{align}

Note that the asymptotes of this hyperbola correspond to the locus of points for which the value of the first eigenvalue is exactly zero.

#### Discussion

Suppose $U_{GA}^* = \bar{U}_{GA}^*$, then both of the sets of inequalities will hold for any value of $U_{gA}^*$. This implies that the eigenvalue will be negative for any value of $U_{gA}^*$.

In [83]:
# vectorized numeric repr for the eigenvalue
_numeric_e1 = sym.lambdify((UGA_star, UGA_prime_star, T, R, P, S, M), e1, modules="numpy")

In [84]:
def plot_first_eigenvalue(T, R, P, S, M):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{GA}^{'*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_1$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100).reshape(-1, 1)
    equilibrium_selection_derivs = np.linspace(0, 10, 1000).reshape(1, -1) # sensible UGA'(0) >= 1
    Z = _numeric_e1(equilibrium_selection_probs, equilibrium_selection_derivs, T, R, P, S, M)
    cax = ax.imshow(Z, origin="lower", aspect="auto", vmin=-0.5, vmax=1.0)

    levels = np.arange(-0.25, 1.25, 0.25)
    contours = ax.contour(Z, levels=levels, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 10, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [85]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")

w = widgets.interactive(plot_first_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider)
display(w)

A Jupyter Widget

In [51]:
UGA_prime_star, = sym.solve(numerator, UGA_prime_star)

In [52]:
sym.factor(UGA_prime_star)

-(P*UGA_star**2 - 2*P*UGA_star + P + R*UGA_star**2 - S*UGA_star**2 + S*UGA_star - T*UGA_star**2 + T*UGA_star)**2/(2*P*R*UGA_star**2 - 2*P*R*UGA_star - P*S*UGA_star**2 + 2*P*S*UGA_star - P*S - R*T*UGA_star**2)

After a bit of fiddling, we arrive at the same stability condition derived above.

\begin{align}
   % U'_{GA}(x^*) < -\frac{(PU_{GA}(x^*)^2 - 2PU_{GA}(x^*) + P + RU_{GA}(x^*)^2 - SU_{GA}(x^*)^2 + SU_{GA}(x^*) - TU_{GA}(x^*)^2 + TU_{GA}(x^*))^2}{(2PRU_{GA}(x^*)^2 - 2PRU_{GA}(x^*) - PSU_{GA}(x^*)^2 + 2PSU_{GA}(x^*) - PS - RTU_{GA}(x^*)^2)} \\
    U'_{GA}(x^*) < \frac{\bigg(\big((R + P) - (T + S)\big)U_{GA}(x^*)^2 + \big((T + S) - 2P\big)U_{GA}(x^*) + P\bigg)^2}{\big(R(T-2P) + PS\big)U_{GA}(x^*)^2 + 2P(R - S)U_{GA}(x^*) + PS}
\end{align}

In [72]:
e2

(-3*M + 3*P*UGA_star**2 - 6*P*UGA_star - P*UgA_star**2 + 2*P*UgA_star + 2*P + 3*R*UGA_star**2 - R*UgA_star**2 - 3*S*UGA_star**2 + 3*S*UGA_star + S*UgA_star**2 - S*UgA_star - 3*T*UGA_star**2 + 3*T*UGA_star + T*UgA_star**2 - T*UgA_star + m)/(4*M - 4*P*UGA_star**2 + 8*P*UGA_star - 4*P - 4*R*UGA_star**2 + 4*S*UGA_star**2 - 4*S*UGA_star + 4*T*UGA_star**2 - 4*T*UGA_star)

$$ e_2 = \left(\frac{\big((R + P) - (T + S)\big)U_{gA}^{*2} + \big((T + S) - 2P\big)U_{gA}^* + P}{4N^*}\right) - \frac{3}{4} $$

### Completing the square

Completing the square for the quadratic polynomial in $U_{gA}(x^*)$ yields the following expression.

\begin{align}
    \big((R + P) - (T + S)\big)\left(U_{gA}^* - \bar{U}_{GA}^*\right)^2 + \frac{1}{2}\bar{N}^* \\
\end{align}

Finally we need to find conditions under which the third eigenvalue is strictly negative.

In [78]:
# vectorized numeric repr for the eigenvalue
_numeric_e2 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, M, m), e2, modules="numpy")

In [79]:
def plot_second_eigenvalue(T, R, P, S, M, m):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_2$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e2(UGAs, UgAs, T, R, P, S, M, m)
    cax = ax.imshow(Z, origin="lower")
    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [80]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")
m_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$m$")

w = widgets.interactive(plot_second_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider, m=m_slider)
display(w)

A Jupyter Widget

In [81]:
e3

(-M + P*UGA_star**2 - 2*P*UGA_star - P*UgA_star**2 + 2*P*UgA_star + R*UGA_star**2 - R*UgA_star**2 - S*UGA_star**2 + S*UGA_star + S*UgA_star**2 - S*UgA_star - T*UGA_star**2 + T*UGA_star + T*UgA_star**2 - T*UgA_star + m)/(2*M - 2*P*UGA_star**2 + 4*P*UGA_star - 2*P - 2*R*UGA_star**2 + 2*S*UGA_star**2 - 2*S*UGA_star + 2*T*UGA_star**2 - 2*T*UGA_star)

In [86]:
# vectorized numeric repr for the eigenvalue
_numeric_e3 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, M, m), e3, modules="numpy")

In [89]:
def plot_third_eigenvalue(T, R, P, S, M, m):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_3$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e3(UGAs, UgAs, T, R, P, S, M, m)
    cax = ax.imshow(Z, origin="lower")
    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [90]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")
m_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$m$")

w = widgets.interactive(plot_third_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider, m=m_slider)
display(w)

A Jupyter Widget

## When is the fitness-maximizing interior equilibrium un-invadable by randomista?

In [147]:
evaluated_e3 = e3.subs({UGA_star: optimal_UGA_star, UgA_star: optimal_x1_star, m: 0})
simplified_e3 = sym.factor(sym.cancel(sym.together(evaluated_e3)))

In [148]:
numerator, denominator = sym.fraction(simplified_e3)

In [151]:
numerator

-4.0*M**3*P**3 - 12.0*M**3*P**2*R + 12.0*M**3*P**2*S + 12.0*M**3*P**2*T - 12.0*M**3*P*R**2 + 24.0*M**3*P*R*S + 24.0*M**3*P*R*T - 12.0*M**3*P*S**2 - 24.0*M**3*P*S*T - 12.0*M**3*P*T**2 - 4.0*M**3*R**3 + 12.0*M**3*R**2*S + 12.0*M**3*R**2*T - 12.0*M**3*R*S**2 - 24.0*M**3*R*S*T - 12.0*M**3*R*T**2 + 4.0*M**3*S**3 + 12.0*M**3*S**2*T + 12.0*M**3*S*T**2 + 4.0*M**3*T**3 + 8.0*M**2*P**3*R + 16.0*M**2*P**2*R**2 - 16.0*M**2*P**2*R*S - 16.0*M**2*P**2*R*T - 2.0*M**2*P**2*S**2 - 4.0*M**2*P**2*S*T - 2.0*M**2*P**2*T**2 + 8.0*M**2*P*R**3 - 16.0*M**2*P*R**2*S - 16.0*M**2*P*R**2*T + 4.0*M**2*P*R*S**2 + 8.0*M**2*P*R*S*T + 4.0*M**2*P*R*T**2 + 4.0*M**2*P*S**3 + 12.0*M**2*P*S**2*T + 12.0*M**2*P*S*T**2 + 4.0*M**2*P*T**3 - 2.0*M**2*R**2*S**2 - 4.0*M**2*R**2*S*T - 2.0*M**2*R**2*T**2 + 4.0*M**2*R*S**3 + 12.0*M**2*R*S**2*T + 12.0*M**2*R*S*T**2 + 4.0*M**2*R*T**3 - 2.0*M**2*S**4 - 8.0*M**2*S**3*T - 12.0*M**2*S**2*T**2 - 8.0*M**2*S*T**3 - 2.0*M**2*T**4 - 4.0*M*P**3*R**2 - 4.0*M*P**2*R**3 + 4.0*M*P**2*R**2*S + 4.0*M*P*

\begin{align}
    4\bigg(-\big(P^3 + 3P^2R + 3PR^2 + R^3\big) - \big(P^3 - 3P^2S + 3PS^2 + S^3\big) + \big(-P^3 + 3P^2T - 3PT^2 + T^3\big) - \big(R^3 - 3R^2S + 3RS^2 - S^3\big) + \big(T^3 - 3RT^2 + 3R^2T - R^3\big) + 2P^3 + 2R^3 + 8PRS + 8PRT - 8PST - 8RST - 2T^3 + \big(S^3 + 3S^2T + 3ST^2 + T^3\big)\bigg)M^3 + 2\big(4P^3R + 8P^2R^2 - 8P^2RS - 8P^2RT - P^2S^2 - 2P^2ST - P^2T^2 + 4PR^3 - 8PR^2S - 8PR^2T + 2PRS^2 + 4PRST + 2PRT^2 + 2PS^3 + 6PS^2T + 6PST^2 + 2PT^3 - R^2S^2 - 2R^2ST - R^2T^2 + 2RS^3 + 6RS^2T + 6RST^2 + 2RT^3 - S^4 - 4S^3T - 6S^2T^2 - 4ST^3 - T^4\big)M^2 + \big(- 4P^3R^2 - 4P^2R^3 + 4P^2R^2S + 4P^2R^2T + 2P^2RS^2 + 4P^2RST + 2P^2RT^2 + 2PR^2S^2 + 4PR^2ST + 2PR^2T^2 - 2PRS^3 - 6PRS^2T - 6PRST^2 - 2PRT^3 - 0.25PS^4 -PS^3T - 1.5PS^2T^2 - PST^3 - 0.25PT^4 - 0.25RS^4 - RS^3T - 1.5RS^2T^2 - RST^3 - 0.25RT^4 + 0.25S^5 + 1.25S^4T + 2.5S^3T^2 + 2.5S^2T^3 + 1.25ST^4 + 0.25T^5\big)M + \big(- P^2R^2S^2 + 2P^2R^2ST - P^2R^2T^2 + P^2RS^3 - P^2RS^2T - P^2RST^2 + P^2RT^3 - 0.25P^2S^4 + 0.5P^2S^2T^2 - 0.25P^2T^4 + PR^2S^3 - PR^2S^2T - PR^2ST^2 + PR^2T^3 - PRS^4 + 2PRS^2T^2 - PRT^4 + 0.25PS^5 + 0.25PS^4T - 0.5PS^3T^2 - 0.5PS^2T^3 + 0.25PST^4 + 0.25PT^5 - 0.25R^2S^4 + 0.5R^2S^2T^2 - 0.25R^2T^4 + 0.25RS^5 + 0.25RS^4T - 0.5RS^3T^2 - 0.5RS^2T^3 + 0.25RST^4 + 0.25RT^5 - 0.0625S^6 - 0.125S^5T + 0.0625S^4T^2 + 0.25S^3T^3 + 0.0625S^2T^4 - 0.125ST^5 - 0.0625T^6\big) \\
    4\bigg(-\big(R + P\big)^3 - \big(P - S\big)^3 + \big(T - P\big)^3 - \big(R - S\big)^3 + \big(T - R\big)^3 + 2P^3 + 2R^3 + 8PRS + 8PRT - 8PST - 8RST - 2T^3 + \big(T + S\big)^3\bigg)M^3 + 2\big(4P^3R + 8P^2R^2 - 8P^2RS - 8P^2RT - P^2S^2 - 2P^2ST - P^2T^2 + 4PR^3 - 8PR^2S - 8PR^2T + 2PRS^2 + 4PRST + 2PRT^2 + 2PS^3 + 6PS^2T + 6PST^2 + 2PT^3 - R^2S^2 - 2R^2ST - R^2T^2 + 2RS^3 + 6RS^2T + 6RST^2 + 2RT^3 - S^4 - 4S^3T - 6S^2T^2 - 4ST^3 - T^4\big)M^2 + \big(- 4P^3R^2 - 4P^2R^3 + 4P^2R^2S + 4P^2R^2T + 2P^2RS^2 + 4P^2RST + 2P^2RT^2 + 2PR^2S^2 + 4PR^2ST + 2PR^2T^2 - 2PRS^3 - 6PRS^2T - 6PRST^2 - 2PRT^3 - 0.25PS^4 -PS^3T - 1.5PS^2T^2 - PST^3 - 0.25PT^4 - 0.25RS^4 - RS^3T - 1.5RS^2T^2 - RST^3 - 0.25RT^4 + 0.25S^5 + 1.25S^4T + 2.5S^3T^2 + 2.5S^2T^3 + 1.25ST^4 + 0.25T^5\big)M + \big(- P^2R^2S^2 + 2P^2R^2ST - P^2R^2T^2 + P^2RS^3 - P^2RS^2T - P^2RST^2 + P^2RT^3 - 0.25P^2S^4 + 0.5P^2S^2T^2 - 0.25P^2T^4 + PR^2S^3 - PR^2S^2T - PR^2ST^2 + PR^2T^3 - PRS^4 + 2PRS^2T^2 - PRT^4 + 0.25PS^5 + 0.25PS^4T - 0.5PS^3T^2 - 0.5PS^2T^3 + 0.25PST^4 + 0.25PT^5 - 0.25R^2S^4 + 0.5R^2S^2T^2 - 0.25R^2T^4 + 0.25RS^5 + 0.25RS^4T - 0.5RS^3T^2 - 0.5RS^2T^3 + 0.25RST^4 + 0.25RT^5 - 0.0625S^6 - 0.125S^5T + 0.0625S^4T^2 + 0.25S^3T^3 + 0.0625S^2T^4 - 0.125ST^5 - 0.0625T^6\big) \\
\end{align}

In [ ]:
(S - R)(S^2 - 2SR + R^2) = S^3 - 3RS^2 + 3SR^2 - R^3

In [158]:
sym.factor(-P**3 - 3*P**2*R + 3*P**2*S + 3*P**2*T - 3*P*R**2 + 8*P*R*S + 8*P*R*T - 3*P*S**2 - 8*P*S*T - 3*P*T**2 - R**3 + 3*R**2*S + 3*R**2*T - 3*R*S**2 - 8*R*S*T - 3*R*T**2 + S**3 + 3*S**2*T + 3*S*T**2 + T**3)

-P**3 - 3*P**2*R + 3*P**2*S + 3*P**2*T - 3*P*R**2 + 8*P*R*S + 8*P*R*T - 3*P*S**2 - 8*P*S*T - 3*P*T**2 - R**3 + 3*R**2*S + 3*R**2*T - 3*R*S**2 - 8*R*S*T - 3*R*T**2 + S**3 + 3*S**2*T + 3*S*T**2 + T**3

In [164]:
r1, r2, r3 = sym.solve(numerator, M)

In [ ]:
sym.cancel(sym.together(r1))

In [149]:
denominator

(0.5*M*P + 0.5*M*R - 0.5*M*S - 0.5*M*T - 0.5*P*R + 0.125*S**2 + 0.25*S*T + 0.125*T**2)**3

Denominator will be negative if and only if...

\begin{align}
    % 4MP + 4MR - 4MS - 4MT - 4PR + S^2 + 2ST + T^2 < 0 \\
    % 4M\big((R + P) - (T + S)\big) - 4PR + (T + S)^2< 0 \\
    % 4M\big((R + P) - (T + S)\big) < 4RP - (T + S)^2 \\
    M < \frac{1}{4}\frac{4RP - (T + S)^2}{(R + P) - (T + S)} \\
    M < \frac{RP - \left(\frac{T + S}{2}\right)^2}{(R + P) - (T + S)} \\
\end{align}

...since we are looking at interior equilibrium the RHS of the above inequality will be positive.

In [139]:
_f = sym.lambdify((T, R, P, S, M), evaluated_e3, modules = "numpy")

def plot_non_invadable_by_randomista(T, R, P, S):
    fig, ax = plt.subplots(1, 1, figsize=(10,6))
    Ms = np.linspace(0, min(0.5 * (T + S), P), 100)
    ax.plot(Ms, _f(T, R, P, S, Ms))
    ax.set_xlabel(r"$M$", fontsize=15)
    ax.set_ylabel(r"$e_3$", rotation="horizontal", fontsize=15)
    plt.show()

In [140]:
# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

w = widgets.interactive(plot_non_invadable_by_randomista, T=T_slider, R=R_slider, P=P_slider, S=S_slider)
display(w)

A Jupyter Widget

In [157]:
solutions = sym.solve(simplified_e3, M)

KeyboardInterrupt: 